<a href="https://colab.research.google.com/github/AnikaZN/DS-Unit-2-Linear-Models/blob/master/Linear_Models_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data

In [45]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/nyc/nyc-rent-2016.csv')
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,exclusive,terrace,loft,garden_patio,common_outdoor_space,wheelchair_access
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df['outdoor'] = (df['roof_deck'] + df['outdoor_space'] + df['balcony'] + df['swimming_pool'] +
                 df['terrace'] + df['loft'] + df['garden_patio'] + df['common_outdoor_space'])

# Train/Test Split

In [47]:
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)
df['created'].head()

0   2016-06-24 07:54:24
1   2016-06-12 12:19:27
2   2016-04-17 03:26:41
3   2016-04-18 02:22:02
4   2016-04-28 01:32:41
Name: created, dtype: datetime64[ns]

In [0]:
df['month'] = df['created'].dt.month

train = df[df['month'] < 6] #April and May
test = df[df['month'] == 6] #June

In [49]:
train.shape, test.shape

((31515, 36), (16785, 36))

In [50]:
train['price'].mean()

3432.7534190068222

# Baseline

In [0]:
from sklearn.metrics import mean_absolute_error as mae

In [52]:
y_test = test['price']
y_pred = [train['price'].mean()] * len(y_test)

print(len(y_test), len(y_pred))
print('Mean Absolute Error:', mae(y_test, y_pred))

16785 16785
Mean Absolute Error: 1052.6798017842948


# Features

In [53]:
df.columns

Index(['bathrooms', 'bedrooms', 'created', 'description', 'display_address',
       'latitude', 'longitude', 'price', 'street_address', 'interest_level',
       'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed',
       'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
       'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck',
       'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony',
       'swimming_pool', 'new_construction', 'exclusive', 'terrace', 'loft',
       'garden_patio', 'common_outdoor_space', 'wheelchair_access', 'outdoor',
       'month'],
      dtype='object')

In [0]:
from sklearn.linear_model import LinearRegression as linreg

In [55]:
model = linreg()
model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
features = ['bedrooms',
           'bathrooms',
           'longitude', #approx 50 points
           'doorman', #approx 12 points
           'laundry_in_unit', #approx 4 points
           'interest_level', #approx 30 points
           'elevator', #approx 3 points
           'outdoor']

target = 'price'

x_train = train[features]
y_train = train[target]

x_test = test[features]
y_test = test[target]

In [57]:
x_train.shape, y_train.shape

((31515, 8), (31515,))

# One-hot Encoding

In [58]:
! pip install category_encoders

In [0]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)

x_train = encoder.fit_transform(x_train)
x_test = encoder.transform(x_test)

In [68]:
model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [69]:
model.coef_

array([ 554.7309774 ,  528.11197038, -362.3518964 ,  188.59280528,
        123.59233485, -110.53090471,  133.75368023,  -75.97962713,
         83.78678701,  -44.87157163])

In [70]:
model.intercept_

3432.7534190069305

# Scale

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [72]:
x_train

array([[-0.45807512, -0.42056347, -0.99016991, ..., -0.55395212,
        -1.0537678 , -0.53702897],
       [-0.45807512, -0.42056347,  0.16078122, ..., -0.55395212,
        -1.0537678 , -0.53702897],
       [ 2.28697859, -0.42056347,  0.7818223 , ..., -0.55395212,
        -1.0537678 , -0.53702897],
       ...,
       [-0.45807512, -0.42056347, -0.59189356, ..., -0.55395212,
         0.94897567, -0.53702897],
       [-1.37309302, -0.42056347, -1.27031344, ..., -0.55395212,
         0.94897567, -0.53702897],
       [ 0.45694278, -0.42056347,  1.86526896, ..., -0.55395212,
        -1.0537678 , -0.53702897]])

# MAE

In [73]:
y_pred = model.predict(x_test)
y_pred

array([3637.10083539, 3826.8070638 , 2438.27610348, ..., 2608.08015382,
       3705.47832827, 3458.31847634])

In [74]:
mae(y_test, y_pred)

600.8393833748322